# Sign Language Recognition Using LSTM DL Model

In [ ]:
!pip install tensorflow==2.5.0 tensorflow-gpu==2.5.0 opencv-python mediapipe sklearn matplotlib

## Installing and importing dependencies

In [130]:
import cv2 as cv
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

## Marking the keypoints

In [131]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [132]:
def mediapipe_det(image, model):
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    image.flags.writeable = False
    res = model.process(image)
    image.flags.writeable = True
    image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
    return image, res

In [133]:
def styled_landmarks(image, model):
    # Face
    mp_drawing.draw_landmarks(image, res.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)) 
    # Pose 
    mp_drawing.draw_landmarks(image, res.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)) 
    # Left hand
    mp_drawing.draw_landmarks(image, res.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)) 
    # Right hand  
    mp_drawing.draw_landmarks(image, res.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)) 

In [ ]:
cap = cv.VideoCapture(0)
# Setting the model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read capture
        ret, frame = cap.read()

        # Detect
        image, res = mediapipe_det(frame, holistic)
        print(res)
        
        # Draw
        styled_landmarks(image, res)

        cv.imshow("Camera Feed", image)

        if cv.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv.destroyAllWindows()

## Extracting the values

In [157]:
pose = []
for resV in res.pose_landmarks.landmark:
    test = np.array([resV.x, resV.y, resV.z, resV.visibility])
    pose.append(test)

In [158]:
def extract(res):
    # 33 * 4 = 33 landmarks, 4 values each (includes resV.visibility)
    pose = np.array([[resV.x, resV.y, resV.z, resV.visibility] for rezV in res.pose_landmarks.landmark]).flatten() if res.pose_landmarks else np.zeros(132)
    # 468 * 3 = 468 landmarks, 3 values each
    face = np.array([[resV.x, resV.y, resV.z] for
    lHand = np.array([[resV.x, resV.y, resV.z] for rezV in res.left_hand_landmarks.landmark]).flatten() if res.left_hand_landmarks else np.zeros(63)
    rHand = np.array([[resV.x, resV.y, resV.z] for rezV in res.right_hand_landmarks.landmark]).flatten() if res.right_hand_landmarks else np.zeros(63) rezV in res.face_landmarks.landmark]).flatten() if res.face_landmarks else np.zeros(1404)
    # 21 * 3 = 21 landmarks, 3 values each
    return np.concatenate([pose, face, lHand, rHand])

## Collection file system

In [184]:
DATA_PATH = os.path.join('MP_Data');
actions = np.array(['hello', "thanks", "i love you"])

# 30 seqences
no_seq = 30

# Every sequence 30 frames long
sequence_length = 30

In [185]:
for action in actions:
    for seq in range(no_seq):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(seq)))
        except:
            pass
